In [1]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
from pathlib import Path
# new_version_path = Path(r'D:\vnpy-2.0.8')
new_version_path = Path(r'E:\vnpy\vnpy-2.0.8')
sys.path.insert(0, str(new_version_path))
# sys.path

import vnpy
print(vnpy.__version__)

2.0.8


In [2]:
import pandas as pd
from datetime import datetime
from vnpy.app.cta_strategy.backtesting import BacktestingEngine
from vnpy.trader.constant import Offset

from utility import comodity_to_vt_symbol, get_output_path
from backtesting import ResearchBacktestingEngine
from trade_match import calculate_trades_result, generate_trade_df, exhaust_trade_result
from turtle_b_strategy import TurtleBStrategy

future_basic_data = pd.read_csv('future_basic_data.csv', index_col=0)

In [3]:
commodity = 'RM'
data_mode = 'main'
interval = '1h'
strategy_name = 'turtle'
strategy_params = {}
capital = 100000
rate = 0
start = datetime(2015, 1, 1)
end = datetime(2019, 12, 1)

vt_symbol = comodity_to_vt_symbol(commodity, 'main')
size = future_basic_data.loc[commodity]['size']
pricetick = future_basic_data.loc[commodity]['pricetick']

engine = ResearchBacktestingEngine()
engine.set_parameters(
    vt_symbol=vt_symbol,
    interval=interval,
    start=start,
    end=end,
    rate=rate,
    slippage=pricetick,
    size=size,
    pricetick=pricetick,
    capital=capital
)
engine.add_strategy(TurtleBStrategy, strategy_params)

# run backtest
engine.load_data()
engine.run_backtesting()

trades = engine.get_all_trades()
last_trade = trades[-1]
if last_trade.offset == Offset.OPEN:
    engine.trades.pop(last_trade.vt_tradeid)
    
pnl_df = engine.calculate_result()
# pnl_df.to_csv(get_output_path('pnl_day_by_day.csv'))
day_res = engine.calculate_statistics()

trades = engine.trades
trade_res = exhaust_trade_result(trades, size, rate, pricetick, capital, show_long_short_condition=False)

d = {
    'day_end_balance': day_res['end_balance'],
    'trade_end_balance': trade_res['end_balance'],
    'day_max_drawdown': day_res['max_drawdown'],
    'trade_max_drawdown': trade_res['max_drawdown'],
    'day_max_ddpercent': day_res['max_ddpercent'],
    'trade_max_ddpercent': trade_res['max_ddpercent'],
    'total_return': day_res['total_return'],
    'annual_return': day_res['annual_return'],
    'sharpe_ratio': day_res['sharpe_ratio'],
    'return_drawdown_ratio': day_res['return_drawdown_ratio'],
    'winning_rate': trade_res['winning_rate'],
    'win_loss_pnl_ratio': trade_res['win_loss_pnl_ratio']
}

In [4]:
d

{'day_end_balance': 93950.0,
 'trade_end_balance': 93950.0,
 'day_max_drawdown': -14450.0,
 'trade_max_drawdown': -13750.0,
 'day_max_ddpercent': -14.145863925599608,
 'trade_max_ddpercent': -13.553474618038441,
 'total_return': -6.05,
 'annual_return': -1.233644859813084,
 'sharpe_ratio': -0.3167137739944342,
 'return_drawdown_ratio': -0.4276868512110727,
 'winning_rate': 0.2857142857142857,
 'win_loss_pnl_ratio': 2.11151073005267}

In [ ]:
# save trade
trades = engine.trades

df = calculate_trades_result(trades)
df
# df.head()

In [ ]:
df2 = generate_trade_df(trades, size, 0.0001, pricetick, capital)
df2